In [2]:
import os

os.chdir("..")
print(os.getcwd())
current_dir = os.getcwd()

/Users/hussain/Menteru-Github/takenaka-wellness-project/takenaka_wellness


In [4]:
import pandas as pd
import psycopg2
from data_module.db_utils.db_host_tables import HOST_TABLES
from data_module.db_utils.private_info import DB_CONFIGS
from psycopg2 import errors, sql
from pathlib import Path
import pandas as pd
import os
from typing import List, Tuple, Generator

In [5]:
print(HOST_TABLES)
print(DB_CONFIGS)

{'host1': ['P00043145A_TBL', 'P00043147A_TBL', 'P00045031A_TBL', 'P00045044A_TBL', 'P00045057A_TBL', 'P00045070A_TBL', 'P00045083A_TBL', 'P00045096A_TBL', 'P00045109A_TBL', 'P00045122A_TBL', 'P00045135A_TBL', 'P00045148A_TBL', 'P00045161A_TBL', 'P00045174A_TBL', 'P00045187A_TBL', 'P00045200A_TBL', 'P00045213A_TBL', 'P00045226A_TBL', 'P00045369A_TBL', 'P00045382A_TBL', 'P00045395A_TBL', 'P00045408A_TBL', 'P00045421A_TBL', 'P00045434A_TBL', 'P00045447A_TBL', 'P00045460A_TBL', 'P00045473A_TBL', 'P00045486A_TBL', 'P00045499A_TBL', 'P00045512A_TBL', 'P00045525A_TBL', 'P00045538A_TBL', 'P00045551A_TBL', 'P00045564A_TBL', 'P00045033A_TBL', 'P00045046A_TBL', 'P00045059A_TBL', 'P00045072A_TBL', 'P00045085A_TBL', 'P00045098A_TBL', 'P00045111A_TBL', 'P00045124A_TBL', 'P00045137A_TBL', 'P00045150A_TBL', 'P00045162A_TBL', 'P00045176A_TBL', 'P00045189A_TBL', 'P00045202A_TBL', 'P00045215A_TBL', 'P00045228A_TBL', 'P00045371A_TBL', 'P00045384A_TBL', 'P00045397A_TBL', 'P00045410A_TBL', 'P00045423A_TBL',

In [5]:
def get_sorted_file_paths(root_path, table_name):
    """
    First collect and sort the file paths - this is small data (just paths)
    """
    files_with_dates = []
    print(f"\nProcessing table {table_name}")
    for root, _, files in os.walk(root_path):
        for file in files:
            if file.endswith(".csv") and file.startswith(table_name):
                date_str = os.path.basename(os.path.dirname(os.path.join(root, file)))
                full_path = os.path.join(root, file)
                files_with_dates.append((date_str, full_path))

    # Sort the files by date
    files_with_dates.sort(key=lambda x: x[0])
    return files_with_dates


def process_csv_files(
    sorted_files: List[Tuple[str, str]],
) -> Generator[pd.DataFrame, None, None]:
    """
    Generator that yields dataframes from the csv files

    This function takes a list of tuples, where each tuple contains a date string and a file path.
    It then reads the csv file and returns a dataframe.

    We use yield to avoid loading all the data into memory at once.

    """
    for date_str, file_path in sorted_files:
        try:
            df = pd.read_csv(file_path)
            if df.empty:
                print(f"Empty dataframe for {file_path}")
                continue
            df["period"] = date_str  # Add period information
            print(
                f"Reading {os.path.basename(file_path)} for period {date_str}, shape: {df.shape}"
            )
            yield df
        except Exception as e:
            print(f"Error reading {file_path}: {e}")


def process_one_table(table_name: str, root_path: str) -> pd.DataFrame:
    """
    Process a single table

    This all take all the files for a single table and concatenate them into a single dataframe
    """
    sorted_files = get_sorted_file_paths(root_path, table_name)
    print("sorted_files: ", sorted_files)
    if not sorted_files:
        print(f"No files found for table {table_name}")
        return None

    dfs = []
    chunk_size = 3  # Process 3 files at a time
    current_chunk = []

    for df in process_csv_files(sorted_files):
        current_chunk.append(df)

        # If the chunk is full, concatenate and save
        if len(current_chunk) >= chunk_size:
            chunk_df = pd.concat(current_chunk, ignore_index=True)
            dfs.append(chunk_df)
            current_chunk = []  # Reset the chunk
            print(f"Processed chunk of {chunk_size} files for {table_name}")

    # Process any remaining files
    if current_chunk:
        chunk_df = pd.concat(current_chunk, ignore_index=True)
        dfs.append(chunk_df)

    # Final concatenation of all chunks
    if dfs:
        final_df = pd.concat(dfs, ignore_index=True)
        print(f"\nFinal Combined shape for {table_name}: {final_df.shape}")
        return final_df

    return None


def process_all_tables(
    tables_list: list, root_path: str, save_csv: bool = False
) -> dict:
    """
    Process all tables from the HOST_TABLES list
    """
    print("tables_list: ", tables_list)
    print("root_path: ", root_path)
    results = {}
    # Process all tables
    for table_name in tables_list:
        print(f"\nStarting processing for table: {table_name}")
        result_df = process_one_table(table_name, root_path)
        if result_df is not None:
            results[table_name] = result_df

    if save_csv:
        for table_name, df in results.items():
            final_df = pd.concat([results[table_name], df], ignore_index=True)
            final_df = final_df.drop(columns=["period"])
            final_df.to_csv(
                current_dir + "/temp_output/host1/" + table_name + ".csv",
                index=False,
                encoding="cp932",
            )

    return results


# Usage
root_path = "/Users/hussain/Menteru-Github/takenaka-wellness-project/00_Data/00_RawData/202104-202403A"
host1_tables_name = HOST_TABLES["host1"]
host2_tables_name = HOST_TABLES["host2"]
# Process all tables
process_all_tables(host1_tables_name, root_path, save_csv=True)

tables_list:  ['P00043145A_TBL', 'P00043147A_TBL', 'P00045031A_TBL', 'P00045044A_TBL', 'P00045057A_TBL', 'P00045070A_TBL', 'P00045083A_TBL', 'P00045096A_TBL', 'P00045109A_TBL', 'P00045122A_TBL', 'P00045135A_TBL', 'P00045148A_TBL', 'P00045161A_TBL', 'P00045174A_TBL', 'P00045187A_TBL', 'P00045200A_TBL', 'P00045213A_TBL', 'P00045226A_TBL', 'P00045369A_TBL', 'P00045382A_TBL', 'P00045395A_TBL', 'P00045408A_TBL', 'P00045421A_TBL', 'P00045434A_TBL', 'P00045447A_TBL', 'P00045460A_TBL', 'P00045473A_TBL', 'P00045486A_TBL', 'P00045499A_TBL', 'P00045512A_TBL', 'P00045525A_TBL', 'P00045538A_TBL', 'P00045551A_TBL', 'P00045564A_TBL', 'P00045033A_TBL', 'P00045046A_TBL', 'P00045059A_TBL', 'P00045072A_TBL', 'P00045085A_TBL', 'P00045098A_TBL', 'P00045111A_TBL', 'P00045124A_TBL', 'P00045137A_TBL', 'P00045150A_TBL', 'P00045162A_TBL', 'P00045176A_TBL', 'P00045189A_TBL', 'P00045202A_TBL', 'P00045215A_TBL', 'P00045228A_TBL', 'P00045371A_TBL', 'P00045384A_TBL', 'P00045397A_TBL', 'P00045410A_TBL', 'P00045423A_T

{'P00043145A_TBL':        point_id          time30  value  alertlevel  period
 0         43145  20210401000000  42.00           0  202104
 1         43145  20210401003000  42.00           0  202104
 2         43145  20210401010000  42.00           0  202104
 3         43145  20210401013000  41.77           0  202104
 4         43145  20210401020000  41.20           0  202104
 ...         ...             ...    ...         ...     ...
 46915     43145  20240331213000  44.67           0  202403
 46916     43145  20240331220000  45.00           0  202403
 46917     43145  20240331223000  45.00           0  202403
 46918     43145  20240331230000  45.00           0  202403
 46919     43145  20240331233000  45.00           0  202403
 
 [46920 rows x 5 columns],
 'P00043147A_TBL':        point_id          time30  value  alertlevel  period
 0         43147  20210401000000  42.00           0  202104
 1         43147  20210401003000  42.00           0  202104
 2         43147  20210401010000  4

In [6]:
import logging
import os
import time
import psycopg2
import psycopg2.extras
from psycopg2 import sql
import pandas as pd
import pandas as pd
import psycopg2
from psycopg2 import errors, sql

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)


class DatabaseClient:
    def __init__(
        self,
        db_configs: dict,
        start_date: str,
        end_date: str,
        datetime_column: str = "time",
        output_path: str = "00_Data",
    ):
        """
        This class is designed to handle connections to one or multiple PostgreSQL databases,
        specified by a dictionary of configurations. It allows querying data within a
        specified date range and saving the results to a designated output path.
        The datetime column used for filtering the data can also be specified.

        Args:
            db_configs: A dictionary mapping host keys to their corresponding database configurations.
            start_date: The start date for the query.
            end_date: The end date for the query.
            datetime_column: The datetime column for the query.
            output_path: The path to save the data.
        """
        self.db_configs = db_configs
        self.start_date = start_date
        self.end_date = end_date
        self.datetime_column = datetime_column
        self.output_path = output_path

    def connect(self, host: str):
        """Establish and return a connection to the PostgreSQL database for the given host.

        Args:
            host: The key identifying which database configuration to use.

        Returns:
            A psycopg2 connection object.
        """
        db_config = self.db_configs.get(host)
        if not db_config:
            logger.error(f"No database configuration found for host '{host}'")
            raise
        try:
            connection = psycopg2.connect(**db_config)
            return connection
        except psycopg2.Error as error:
            logger.error(f"Error connecting to host '{host}': {error}")
            raise

    def get_file_paths(self, root_path: str) -> list:
        """
        Collect and the file paths, results is a list of paths for all the csv files in the root path

        """
        from pathlib import Path

        data_full_paths = []
        root = Path(root_path)
        data_full_paths = [str(root / file) for file in root.glob("*.csv")]

        print("data_full_paths:  ", data_full_paths)
        return data_full_paths

    def read_csv_files(
        self,
        file_paths: str,
    ) -> Generator[pd.DataFrame, None, None]:
        """
        Generator that yields dataframes from the csv files

        This function takes a list of tuples, where each tuple contains a date string and a file path.
        It then reads the csv file and returns a dataframe.

        We use yield to avoid loading all the data into memory at once.

        """
        data = {}
        for file_path in file_paths:
            print(f"Reading heree {os.path.basename(file_path)}")
            try:
                # Read CSV first with all columns as objects/strings
                df = pd.read_csv(file_path)

                # Print data info for debugging
                print(f"Initial data types: {df.dtypes}")
                print(f"NaN values in value column: {df['value'].isna().sum()}")
                print(
                    f"NaN values in alertlevel column: {df['alertlevel'].isna().sum()}"
                )
                # First convert to string
                df["time"] = df["time"].astype(str)
                # Remove decimal point and any following digits
                df["time"] = df["time"].str.split(".").str[0]
                # When reading CSV files
                df["time"] = df["time"].replace("nan", None)  # Replace 'nan' with NULL
                df = df.dropna(subset=["time"])  # Drop rows with NULL time values
                # Convert values with proper handling of NaN values
                df["point_id"] = df["point_id"].astype(str)
                df["time"] = df["time"].astype(str)

                # Handle NaN values in numeric columns before conversion
                df["value"] = pd.to_numeric(df["value"], errors="coerce")
                df["value"] = df["value"].fillna(0).astype(int)  # Fill NaN with 0

                df["alertlevel"] = pd.to_numeric(df["alertlevel"], errors="coerce")
                df["alertlevel"] = (
                    df["alertlevel"].fillna(0).astype(int)
                )  # Fill NaN with 0

                if df.empty:
                    print(f"Empty dataframe for {file_path}")
                    continue

                print(
                    f"Successfully read {os.path.basename(file_path)}, shape: {df.shape}"
                )
                yield df, os.path.basename(file_path).split(".")[0]
            except Exception as e:
                print(f"Error reading {file_path}: {e}")
                # Re-raise the exception for better debugging
                raise

    def add_dataframe_to_db(
        self,
        cursor: psycopg2.extensions.cursor,
        host: str,
        df: pd.DataFrame,
        df_name: str,
    ) -> str:
        import psycopg2
        import psycopg2.extras
        from psycopg2 import sql
        import pandas as pd

        print(f"Adding table '{df_name}' and inserting DataFrame data into the db")

        try:
            # If the CSV header is different (e.g., "time30"), rename it.
            if "time30" in df.columns:
                df = df.rename(columns={"time30": "time"})
                print(f"Renamed 'time30' column to 'time' for table {df_name}")

            # Create the table if it doesn't exist.
            create_table_query = sql.SQL(
                """
                CREATE TABLE IF NOT EXISTS {} (
                    point_id VARCHAR(8),
                    time VARCHAR(14) NOT NULL,
                    value INTEGER,        
                    alertlevel INTEGER,
                    PRIMARY KEY (point_id, time)
                )
                """
            ).format(sql.Identifier(df_name))

            # Debug: print the SQL command
            print("Executing create table query:")
            print(create_table_query.as_string(cursor))

            cursor.execute(create_table_query)
            cursor.connection.commit()

            # Check if the table was created by querying the catalog.
            cursor.execute(
                sql.SQL("SELECT to_regclass('public.{}')").format(
                    sql.Identifier(df_name)
                )
            )
            table_exists = cursor.fetchone()[0]
            print(f"Table exists after creation attempt: {table_exists}")

            # Prepare and execute the bulk insert.
            insert_query = sql.SQL(
                "INSERT INTO {} (point_id, time, value, alertlevel) VALUES %s ON CONFLICT DO NOTHING"
            ).format(sql.Identifier(df_name))

            # Convert DataFrame rows into a list of tuples.
            values = list(
                df[["point_id", "time", "value", "alertlevel"]].itertuples(
                    index=False, name=None
                )
            )
            psycopg2.extras.execute_values(cursor, insert_query, values)
            cursor.connection.commit()

            # Verify the number of rows inserted.
            cursor.execute(
                sql.SQL("SELECT COUNT(*) FROM {}").format(sql.Identifier(df_name))
            )
            count = cursor.fetchone()[0]
            print(f"Rows inserted into {df_name}: {count}")

            return f"Finished adding table '{df_name}' and inserting data (rows inserted: {count})"
        except Exception as e:
            cursor.connection.rollback()  # Roll back the failed transaction
            print(f"Error during table creation/insertion: {e}")
            return f"Error: {e}"

    def add_data_to_db(self, root_path: str, host: str) -> dict:
        """add data from multiple tables on the specified host.

        It attempts to connect to the host using the provided configuration.
        Once connected, it iterates over the table list and queries each one.

        Args:
            table_names: A list of table names to add data from.
            host: The host key for which the configuration should be used.

        Returns:
            A dictionary mapping each table name to its corresponding DataFrame.
            Tables that do not exist are skipped.
        """
        # time the process
        connection = self.connect(host)

        get_file_paths = self.get_file_paths(root_path)
        for df, df_name in self.read_csv_files(get_file_paths):
            try:
                with connection.cursor() as cursor:
                    self.add_dataframe_to_db(cursor, host, df, df_name)
                    logger.info(f"Data added for dataframe {df_name}")
                    connection.commit()  # Commit successful transaction
            except Exception as e:
                connection.rollback()  # Ensure transaction is rolled back
                logger.error(f"Failed to add data for dataframe {df}: {e}")
                continue  # Continue with next table even if this one fails

        connection.close()

        return "Finished adding data to db"


# Main script: using a single configuration dictionary for multiple databases
if __name__ == "__main__":
    start_time = time.time()
    db_client = DatabaseClient(DB_CONFIGS, start_date=None, end_date=None)
    host1_tables_path = "/Users/hussain/Menteru-Github/takenaka-wellness-project/takenaka_wellness/notebooks/"
    # host2_tables_path = "/Users/hussain/Menteru-Github/takenaka-wellness-project/00_Data/archive/temp_output/host2/"

    # data_file_paths = db_client.get_file_paths(host1_tables_path)
    db_client.add_data_to_db(root_path=host1_tables_path, host="host1")
    # db_client.add_data_to_db(root_path=host2_tables_path, host="host2")
    # read_data = db_client.read_csv_files(data_file_paths)
    # for df in db_client.read_csv_files(data_file_paths):
    #     print(f"Processing DataFrame with shape: {df.shape}")

    # # Fetch data for all tables on each host
    # data_host1 = db_client.fetch_multiple_tables_data(HOST_TABLES["host1"], "host1")

    end_time = time.time()
    logger.info(f"Time taken to fetch the whole data: {end_time - start_time} seconds")

data_full_paths:   ['/Users/hussain/Menteru-Github/takenaka-wellness-project/takenaka_wellness/notebooks/P00044005A_TBL.csv']
Reading heree P00044005A_TBL.csv
Initial data types: point_id        int64
time          float64
value         float64
alertlevel    float64
dtype: object
NaN values in value column: 2
NaN values in alertlevel column: 2
Successfully read P00044005A_TBL.csv, shape: (2754248, 4)
Adding table 'P00044005A_TBL' and inserting DataFrame data into the db
Executing create table query:

                CREATE TABLE IF NOT EXISTS "P00044005A_TBL" (
                    point_id VARCHAR(8),
                    time VARCHAR(14) NOT NULL,
                    value INTEGER,        
                    alertlevel INTEGER,
                    PRIMARY KEY (point_id, time)
                )
                
Table exists after creation attempt: "P00044005A_TBL"


INFO:__main__:Data added for dataframe P00044005A_TBL
INFO:__main__:Time taken to fetch the whole data: 84.07258868217468 seconds


Rows inserted into P00044005A_TBL: 1377122
